<a href="https://colab.research.google.com/github/Ayanlola2002/DATA-SCIENCE-PROJECTS/blob/master/NeedEnergyAPI_Data_Wrangling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [86]:
#mounting Gdrive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [87]:
import pandas as pd
import numpy as np
from datetime import date, datetime
import holidays

In [88]:
df_5min=pd.read_csv('/content/drive/MyDrive/NEEDENERGY/data_five_min_id_47803.csv')
df_hourly=pd.read_csv('/content/drive/MyDrive/NEEDENERGY/data_hourly_id_47803.csv')
df_daily=pd.read_csv('/content/drive/MyDrive/NEEDENERGY/data_daily_id_47803.csv')
df_monthly=pd.read_csv('/content/drive/MyDrive/NEEDENERGY/data_monthly_id_47803.csv')
df_public_holiday=pd.read_csv('/content/drive/MyDrive/NEEDENERGY/public_holidays_weekends_ZIM.csv')


In [89]:
##Wrangling Helper Functions to Use
"""
This function do the reading of data file from file path(i.e drive)
"""
def do_read_data(datafile, dateCol):
    ''' Read the excel file'''
    df = pd.read_excel(datafile, index_col= dateCol, parse_dates=True)
    return df

"""
This function Shows Information of the Data file
Like The Header columns, Dimension of Data
"""

def do_data_information(datafile, info = False, head = True, shape = True):
    ''' Look at the info of the data'''
    
    if not isinstance(datafile, pd.DataFrame):
        df = read_data(datafile)
    else:
        df = datafile
    
    ## Information of data
    if info:
        df.info()
        
    print('--------------------------------------------------------------------')
    print('First rows')
    # Show first rows
    if head:
        print(df.head())
        
    print('--------------------------------------------------------------------')
    print('Dimension of the data')
    ## Dimension of data
    if shape:
        print(f'This data has {df.shape[0]} rows and {df.shape[1]} variables') 
        
###This function returns the number of distinct values from any columns       
def do_unique_values(df, col):
    ''' Find the unique values of a columns'''
    return df[col].nunique()


###This function statistically describe the data values
def do_description(df, col = None):
    ''' Give the statistic summary'''
    
    ## Summary statistic
    if col is not None:
        stat = df[col].describe()
    else:
        stat = df.describe()

    ## Change the name of the 50% index to median
    idx = stat.index.tolist()
    idx[5] = 'median'
    stat.index = idx
    return stat

###This function finds % Missing values
def check_missing_value(data):
    ## percentage of missing values
    n = data.isnull().sum().sort_values(ascending=False)/ len(data) * 100
    return n

def check_missing_values_table(df):
  mis_val = df.isnull().sum()
  mis_val_percent = 100 * df.isnull().sum() / len(df)
  mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
  mis_val_table_ren_columns = mis_val_table.rename(
  columns = {0 : 'Missing Values', 1 : '% of Total Values'})
  mis_val_table_ren_columns = mis_val_table_ren_columns[
      mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
  '% of Total Values', ascending=False).round(1)
  print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
      "There are " + str(mis_val_table_ren_columns.shape[0]) +
        " columns that have missing values.")
  return mis_val_table_ren_columns


###This function finds the amount of each values in a dataset
def do_valueCount(data, col, perc = False):
    ''' calculate the proportion in fraction or percentage'''
    if not perc:
      return data[col].value_counts()
    else:
      return data[col].value_counts(normalize = True).mul(100).round(1).astype(str) + '%'    
### This function count number of unique values for all columns in a dataset
def do_nunique_value(df):
  print('The Numver of unique values per columns are as follows')
  print(df.agg(['nunique']).T)


###For date time conversion start here   
def do_date_col_convertion(df, column_name):
  df[column_name]=pd.to_datetime(df[column_name])
  return df[column_name]

def do_time_col_convertion(df, column_name):
    return pd.to_timedelta(df[column_name])


def do_datepart_extraction(data):
  data['date'] = pd.to_datetime(data['date'])
  data['dateOnly']=data['date'].dt.date
  data['year'] = data['date'].dt.year
  data['quarter']=data['date'].dt.quarter
  data['month'] = data['date'].dt.month
  data['weekday'] = data['date'].dt.weekday
  data['day_name'] = data['date'].dt.day_name()
  data['dayofyear'] = data['date'].dt.dayofyear
  data['day'] =data['date'].dt.day
  data['hour'] = data['date'].dt.hour
  data['minute'] =data['date'].dt.minute
  #data['Season']=data['month'].apply(do_year_season)
  data['season'] = data.date.map(do_season_of_date)
  data['day_section']=data['hour'].apply(do_day_sections)
  return data
   

def do_season_of_date(date):
  year = str(date.year)
  seasons = {'spring': pd.date_range(start='21/03/'+year, end='20/06/'+year),
              'summer': pd.date_range(start='21/06/'+year, end='22/09/'+year),
              'autumn': pd.date_range(start='23/09/'+year, end='20/12/'+year)}
  if date in seasons['spring']:
    return 'spring'
  if date in seasons['summer']:
    return 'summer'
  if date in seasons['autumn']:
    return 'autumn'
  else:
    return 'winter'

# Assuming df has a date column of type `datetime`
#df['season'] = df.date.map(season_of_date)


def do_day_sections(x):
  if (x > 4) and (x <= 8):
    return 'Early Morning'
  elif (x > 8) and (x <= 12 ):
    return 'Morning'
  elif (x > 12) and (x <= 16):
    return'Noon'
  elif (x > 16) and (x <= 20) :
    return 'Evening'
  elif (x > 20) and (x <= 24):
    return'Night'
  elif (x <= 4):
    return'Late Night'




def get_holidays(startYear = 2018, endYear = 2025, countryCode = 'ZA'):
  """
    Takes in a start and end date, and start and end year.
    
    Produces a dataframe with a daily date and columns:
    holiday - 'Y' for holiday
    holidayName - name of the holiday if holiday is 'Y'
    
    Returns a dataframe
  """
  holidayDict = {}
  for i in range(startYear, endYear):
    for date, name in sorted(holidays.CountryHoliday(countryCode,years=[i]).items()):
      holidayDict[date] = name
      holiday_df = pd.DataFrame(list(holidayDict.items()),columns = ['day','holidayName'])
      holiday_df['day'] = pd.to_datetime(holiday_df['day'], format='%Y-%M-%d').dt.date
    return holiday_df
    

    
def get_days(start = '1/1/2018',startYear = 2018, end = '31/12/2025',endYear = 2025, countryCode = 'ZA'):
    """
    Takes in a start and end date, and start and end year.
    
    Produces a dataframe with a daily date and columns:
    weekend - 'Y' for weendend and 'N' for workday
    dayOfweek - numerical day of the week identifier 0 for monday
    weekNum - numerical number of the week 
    holiday - 'Y' for holiday
    holidayName - name of the holiday if holiday is 'Y'
    
    Returns a dataframe
    """
    
    #generate the range of daily dates
    dates = pd.date_range(start = start, end = end)
    date_df = pd.DataFrame(dates, columns = ['day'])
    date_df['day'] = pd.to_datetime(date_df['day'], format='%Y-%M-%d')
    country_holidays = get_holidays(startYear = startYear, endYear = endYear, countryCode = countryCode)
    
    date_df['dayName'] = pd.DatetimeIndex(date_df['day']).day_name()
    date_df['dayOfWeek'] = date_df['day'].dt.dayofweek
    date_df['weekend'] = date_df['dayOfWeek'].apply(lambda x: 'Y' if x>4 else 'N')
    date_df['weekNum'] = date_df['day'].dt.week
    date_df['holiday'] = date_df['day'].apply(lambda x: 'Y' if x in country_holidays['day'].values else 'N')
    date_df['day'] = date_df['day'].dt.date
    date_df = date_df.merge(country_holidays, on='day', how='left', indicator=False)
    
    date_df.to_csv(f'../data/public_holidays_weekends.csv', index=False)
    return date_df

###Functions to impute nan values for numerical varriable
def Impute_nan(df):
  numericalcol = [f for f in df_5min.columns if df_5min.dtypes[f] != 'object']
  missing_col=df_5min[numericalcol].columns[df_5min[numericalcol].isnull().any()].tolist()
  #Technique 1: Using mean to impute the missing values
  for i in missing_col:
    df.loc[df.loc[:,i].isnull(),i]=df.loc[:,i].mean()

def Impute_na(df):
  numericalcol = [f for f in df_5min.columns if df_5min.dtypes[f] != 'object']
  missing_col=df_5min[numericalcol].columns[df_5min[numericalcol].isnull().any()].tolist()
  #Technique 1: Using mean to impute the missing values
  for i in missing_col:
     df[i].fillna(mean,inplace=True)


dataList=[df_5min,df_hourly,df_daily,df_monthly]

def do_wrangling_task(dL):
  for i in range(len(dL)):
    print(do_data_information(dL[i]))
    print('===============================================')
    print(check_missing_values_table(dL[i]))
    print('===============================================')
    print(do_nunique_value(dL[i]))
    print('===============================================')
    #Feature Engineering
    dL[i]=do_datepart_extraction(dL[i])
    print('===============================================')
    dL[i].date.min(),dL[i].date.max()
    print('===============================================')
    dL[i].consumption.min(),dL[i].consumption.max()
    print('===============================================')
    #checking for days where there weere no consumptions
    dL[i][dL[i].consumption==0].value_counts().count(),dL[i][dL[i].consumption==0].value_counts()
    print('===============================================')
    dL[i]['Public_Holiday'] = dL[i]['dateOnly'].map(mapping)
    print('===============================================')
    dL[i]['Public_Holiday']=dL[i]['Public_Holiday'].apply(lambda x:np.NaN if isinstance(x, str) and (x.isspace() or not x) else x).replace(np.NaN,'Non_Holiday')
    print('===============================================')
    dL[i]=dL[i][['date','timestamp','year','quarter','month','dayofyear','day','day_name','hour','minute','season','day_section','Public_Holiday','consumption','solar','alwaysOn','gridImport','gridExport','selfConsumption','selfSufficiency','active','reactive','voltages','phaseVoltages','currentHarmonics','voltageHarmonics']]
    print('===============================================')
    print('===============================================')



#df['holidays']=df['date'].apply(lambda(x: 'Hols' if x in get_holidays()['day'].values else 'Non-Hols'))



In [91]:
#Data Description and Information for df_5min
print(do_data_information(df_5min))
print(do_data_information(df_monthly))
print(do_data_information(df_public_holiday))

--------------------------------------------------------------------
First rows
                  date  ...                                   voltageHarmonics
0  2020-03-15 18:15:00  ...  [[], [], [], [], [], [], [], [], [], [], [], [...
1  2020-03-15 18:20:00  ...  [[], [], [], [], [], [], [], [], [], [], [], [...
2  2020-03-15 18:25:00  ...  [[], [], [], [], [], [], [], [], [], [], [], [...
3  2020-03-15 18:30:00  ...  [[], [], [], [], [], [], [], [], [], [], [], [...
4  2020-03-15 18:35:00  ...  [[], [], [], [], [], [], [], [], [], [], [], [...

[5 rows x 15 columns]
--------------------------------------------------------------------
Dimension of the data
This data has 95649 rows and 15 variables
None
--------------------------------------------------------------------
First rows
         date  ...                                   voltageHarmonics
0  2020-03-01  ...  [[], [], [], [], [], [], [], [], [], [], [], [...
1  2020-04-01  ...  [[], [], [], [], [], [], [], [], [], [], [], 

Data Quality Information:
df_5min : The data has 95649 rows and 15 features

In [92]:
#Checking Missing Values by percentage
print(check_missing_values_table(df_5min))
print(check_missing_values_table(df_public_holiday))


Your selected dataframe has 15 columns.
There are 0 columns that have missing values.
Empty DataFrame
Columns: [Missing Values, % of Total Values]
Index: []
Your selected dataframe has 7 columns.
There are 0 columns that have missing values.
Empty DataFrame
Columns: [Missing Values, % of Total Values]
Index: []


In [94]:
#Checking number of Unique values
print(do_nunique_value(df_5min))
print('======================================')
print(do_nunique_value(df_public_holiday))

The Numver of unique values per columns are as follows
                  nunique
date                95649
timestamp           95649
consumption         76377
solar               50268
alwaysOn              955
gridImport          66188
gridExport              1
selfConsumption         2
selfSufficiency      5594
active              95512
reactive            95595
voltages              389
phaseVoltages         389
currentHarmonics        1
voltageHarmonics        1
None
The Numver of unique values per columns are as follows
             nunique
day             2558
dayName            7
dayOfWeek          7
weekend            2
weekNumber        53
holiday            2
holidayName       24
None


In [95]:
#Feature Engineering
#extracting date part
df_5min=do_datepart_extraction(df_5min)


In [96]:
df_5min.head()

,date,timestamp,consumption,solar,alwaysOn,gridImport,gridExport,selfConsumption,selfSufficiency,active,reactive,voltages,phaseVoltages,currentHarmonics,voltageHarmonics,dateOnly,year,quarter,month,weekday,day_name,dayofyear,day,hour,minute,season,day_section
0,2020-03-15 18:15:00,1584288900000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[0.1, 3.7, 0.0, -81.6, -60.0, 30.2, -230.6, -0...","[0.5, 0.9, 0.0, 55.4, 132.4, 14.0, 137.5, 0.2,...","[244.5, None, None]","[244.5, None, None]","[[], [], [], [], [], [], [], [], [], [], [], [...","[[], [], [], [], [], [], [], [], [], [], [], [...",2020-03-15,2020,1,3,6,Sunday,75,15,18,15,winter,Evening
1,2020-03-15 18:20:00,1584289200000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[0.1, 3.8, 0.0, -82.2, -58.5, 30.5, -233.1, -0...","[0.5, 0.9, 0.0, 55.7, 131.9, 14.3, 144.3, 0.2,...","[246.1, None, None]","[246.1, None, None]","[[], [], [], [], [], [], [], [], [], [], [], [...","[[], [], [], [], [], [], [], [], [], [], [], [...",2020-03-15,2020,1,3,6,Sunday,75,15,18,20,winter,Evening
2,2020-03-15 18:25:00,1584289500000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[0.1, 28.5, -26.5, -74.4, -79.2, 55.5, -228.5,...","[0.5, 19.0, 8.8, 79.6, 137.6, 29.4, 175.0, 0.4...","[246.4, None, None]","[246.4, None, None]","[[], [], [], [], [], [], [], [], [], [], [], [...","[[], [], [], [], [], [], [], [], [], [], [], [...",2020-03-15,2020,1,3,6,Sunday,75,15,18,25,winter,Evening
3,2020-03-15 18:30:00,1584289800000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[0.5, 21.3, -18.6, -76.9, -54.7, 48.7, -227.8,...","[2.2, 13.3, 6.5, 72.4, 135.9, 24.8, 152.8, 0.8...","[246.0, None, None]","[246.0, None, None]","[[], [], [], [], [], [], [], [], [], [], [], [...","[[], [], [], [], [], [], [], [], [], [], [], [...",2020-03-15,2020,1,3,6,Sunday,75,15,18,30,winter,Evening
4,2020-03-15 18:35:00,1584290100000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[0.1, 3.7, 0.0, -82.6, -33.2, 30.7, 96.6, 0.2,...","[0.5, 0.9, 0.0, 55.2, 128.1, 13.9, 87.7, 0.7, ...","[246.2, None, None]","[246.2, None, None]","[[], [], [], [], [], [], [], [], [], [], [], [...","[[], [], [], [], [], [], [], [], [], [], [], [...",2020-03-15,2020,1,3,6,Sunday,75,15,18,35,winter,Evening


In [97]:
#showing min and maximm date column values
df_5min.date.min(),df_5min.date.max()

(Timestamp('2020-03-15 18:15:00'), Timestamp('2021-02-12 01:55:00'))

In [98]:
#showing consumption min and max values
df_5min.consumption.min(),df_5min.consumption.max()

(0.0, 939.27)

In [99]:
#checking for date where  consumptions = 0
df_5min[df_5min.consumption==0].value_counts().count(),df_5min[df_5min.consumption==0].value_counts()

(11,
 date                 timestamp      consumption  solar  alwaysOn  gridImport  gridExport  selfConsumption  selfSufficiency  active                                                                                                                                                                       reactive                                                                                                                                                              voltages             phaseVoltages        currentHarmonics                                                                                                  voltageHarmonics                                                                                                  dateOnly    year  quarter  month  weekday  day_name  dayofyear  day  hour  minute  season  day_section
 2020-03-15 19:05:00  1584291900000  0.0          0.0    0.0       0.0         0.0         0.0              0.0              [0.1, 20.2, 0.0, 35.8, 0.0, 46.9,

In [100]:
#checking for days where  consumptions > 0
df_5min[df_5min.consumption>0].value_counts().count(),df_5min[df_5min.consumption>0].value_counts()

(95638,
 date                 timestamp      consumption  solar   alwaysOn  gridImport  gridExport  selfConsumption  selfSufficiency  active                                                                                                                                                               reactive                                                                                                                                                                voltages             phaseVoltages        currentHarmonics                                                                                                  voltageHarmonics                                                                                                  dateOnly    year  quarter  month  weekday  day_name  dayofyear  day  hour  minute  season  day_section  
 2021-02-12 01:55:00  1613087700000  153.934      46.152  1006.212  107.781     0.0         100.0            29.98            [0.1, 3.6, -0.1, None, None, Non

In [101]:
df_public_holiday.head()

,day,dayName,dayOfWeek,weekend,weekNumber,holiday,holidayName
0,2018/01/01,Monday,0,N,1,Y,New Year's Day
1,2018/01/02,Tuesday,1,N,1,N,
2,2018/01/03,Wednesday,2,N,1,N,
3,2018/01/04,Thursday,3,N,1,N,
4,2018/01/05,Friday,4,N,1,N,


In [102]:
df_public_holiday.holidayName.value_counts()

                                         2460
Holy Saturday                               7
Africa Day                                  7
Easter Sunday                               7
Zimbabwe Defence Forces Day                 7
Robert Mugabe National Youth Day            7
Good Friday                                 7
Heroes' Day                                 7
New Year's Day                              6
Christmas Day                               6
Easter Monday                               6
Worker's Day                                6
Independence Day                            5
Boxing Day                                  5
National Unity Day                          5
National Unity Day (observed)               2
Solidarity Day Against Sanctions            1
Christmas Day (observed) / Boxing Day       1
Independence Day (observed)                 1
New Year's Day (observed)                   1
Boxing Day (observed)                       1
Worker's Day (observed)           

In [103]:
df_public_holiday.isnull().sum()

day            0
dayName        0
dayOfWeek      0
weekend        0
weekNumber     0
holiday        0
holidayName    0
dtype: int64

In [104]:
df_public_holiday['day'] = pd.to_datetime(df_public_holiday['day'],format='%Y-%m-%d')

In [106]:
#creating dictionary for mapping hoidays to original table
mapping = dict(df_public_holiday[['day','holidayName']].values)



In [107]:
mapping

{Timestamp('2018-01-01 00:00:00'): "New Year's Day",
 Timestamp('2018-01-02 00:00:00'): ' ',
 Timestamp('2018-01-03 00:00:00'): ' ',
 Timestamp('2018-01-04 00:00:00'): ' ',
 Timestamp('2018-01-05 00:00:00'): ' ',
 Timestamp('2018-01-06 00:00:00'): ' ',
 Timestamp('2018-01-07 00:00:00'): ' ',
 Timestamp('2018-01-08 00:00:00'): ' ',
 Timestamp('2018-01-09 00:00:00'): ' ',
 Timestamp('2018-01-10 00:00:00'): ' ',
 Timestamp('2018-01-11 00:00:00'): ' ',
 Timestamp('2018-01-12 00:00:00'): ' ',
 Timestamp('2018-01-13 00:00:00'): ' ',
 Timestamp('2018-01-14 00:00:00'): ' ',
 Timestamp('2018-01-15 00:00:00'): ' ',
 Timestamp('2018-01-16 00:00:00'): ' ',
 Timestamp('2018-01-17 00:00:00'): ' ',
 Timestamp('2018-01-18 00:00:00'): ' ',
 Timestamp('2018-01-19 00:00:00'): ' ',
 Timestamp('2018-01-20 00:00:00'): ' ',
 Timestamp('2018-01-21 00:00:00'): ' ',
 Timestamp('2018-01-22 00:00:00'): ' ',
 Timestamp('2018-01-23 00:00:00'): ' ',
 Timestamp('2018-01-24 00:00:00'): ' ',
 Timestamp('2018-01-25 00:0

In [108]:
#adding Public holiday column to original
df_5min['Public_Holiday'] = df_5min['dateOnly'].map(mapping)

In [110]:
#replacing nan and empty space with non-Holiday
df_5min['Public_Holiday']=df_5min['Public_Holiday'].apply(lambda x:np.NaN if isinstance(x, str) and (x.isspace() or not x) else x).replace(np.NaN,'Non_Holiday')


In [111]:
df_5min.sample(5)

,date,timestamp,consumption,solar,alwaysOn,gridImport,gridExport,selfConsumption,selfSufficiency,active,reactive,voltages,phaseVoltages,currentHarmonics,voltageHarmonics,dateOnly,year,quarter,month,weekday,day_name,dayofyear,day,hour,minute,season,day_section,Public_Holiday
12957,2020-04-29 19:55:00,1588182900000,286.671,48.958,1369.536,237.712,0.0,100.0,17.08,"[0.1, 3.7, 0.0, None, None, None, None, 0.3, N...","[0.5, 0.9, 0.0, None, None, None, None, 0.2, N...","[245.2, None, None]","[245.2, None, None]","[[], [], [], [], [], [], [], [], [], [], [], [...","[[], [], [], [], [], [], [], [], [], [], [], [...",2020-04-29,2020,2,4,2,Wednesday,120,29,19,55,winter,Evening,Non_Holiday
3635,2020-03-28 10:25:00,1585383900000,184.575,87.091,595.824,97.483,0.0,100.0,47.19,"[0.1, 3.5, 0.0, None, None, None, None, 0.4, N...","[0.4, 0.8, 0.0, None, None, None, None, 0.2, N...","[238.7, None, None]","[238.7, None, None]","[[], [], [], [], [], [], [], [], [], [], [], [...","[[], [], [], [], [], [], [], [], [], [], [], [...",2020-03-28,2020,1,3,5,Saturday,88,28,10,25,winter,Morning,Non_Holiday
70254,2020-11-15 19:20:00,1605460800000,229.366,74.630,532.512,154.736,0.0,100.0,32.54,"[0.1, 3.6, 0.0, None, None, None, None, 0.4, N...","[-0.4, 0.8, 0.0, None, None, None, None, -0.2,...","[240.2, None, None]","[240.2, None, None]","[[], [], [], [], [], [], [], [], [], [], [], [...","[[], [], [], [], [], [], [], [], [], [], [], [...",2020-11-15,2020,4,11,6,Sunday,320,15,19,20,winter,Evening,Non_Holiday
53614,2020-09-19 00:10:00,1600467000000,158.828,61.738,1668.816,97.089,0.0,100.0,38.87,"[0.1, 3.5, 0.0, None, None, None, None, 0.4, N...","[0.4, 0.8, 0.0, None, None, None, None, 0.2, N...","[237.2, None, None]","[237.2, None, None]","[[], [], [], [], [], [], [], [], [], [], [], [...","[[], [], [], [], [], [], [], [], [], [], [], [...",2020-09-19,2020,3,9,5,Saturday,263,19,0,10,winter,Late Night,Non_Holiday
87506,2021-01-14 17:35:00,1610638500000,342.701,53.060,1712.436,289.641,0.0,100.0,15.48,"[0.1, 3.6, 0.0, None, None, None, None, 1.1, N...","[-0.4, 0.9, 0.0, None, None, None, None, -0.4,...","[242.4, None, None]","[242.4, None, None]","[[], [], [], [], [], [], [], [], [], [], [], [...","[[], [], [], [], [], [], [], [], [], [], [], [...",2021-01-14,2021,1,1,3,Thursday,14,14,17,35,winter,Evening,Non_Holiday


In [112]:
df_5min['Public_Holiday'].unique()

array(['Non_Holiday', 'Good Friday', 'Holy Saturday', 'Easter Sunday',
       'Easter Monday', 'Independence Day', "Worker's Day", 'Africa Day',
       "Heroes' Day", 'Zimbabwe Defence Forces Day', 'National Unity Day',
       'Christmas Day', 'Boxing Day', "New Year's Day"], dtype=object)

In [113]:
df_5min.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95649 entries, 0 to 95648
Data columns (total 28 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   date              95649 non-null  datetime64[ns]
 1   timestamp         95649 non-null  int64         
 2   consumption       95649 non-null  float64       
 3   solar             95649 non-null  float64       
 4   alwaysOn          95649 non-null  float64       
 5   gridImport        95649 non-null  float64       
 6   gridExport        95649 non-null  float64       
 7   selfConsumption   95649 non-null  float64       
 8   selfSufficiency   95649 non-null  float64       
 9   active            95649 non-null  object        
 10  reactive          95649 non-null  object        
 11  voltages          95649 non-null  object        
 12  phaseVoltages     95649 non-null  object        
 13  currentHarmonics  95649 non-null  object        
 14  voltageHarmonics  9564

In [115]:
#Re-arranging columns
df_5min=df_5min[['date','timestamp','year','quarter','month','dayofyear','day','day_name','hour','minute','season','day_section','Public_Holiday','consumption','solar','alwaysOn','gridImport','gridExport','selfConsumption','selfSufficiency','active','reactive','voltages','phaseVoltages','currentHarmonics','voltageHarmonics']]

In [116]:
df_5min.head()

,date,timestamp,year,quarter,month,dayofyear,day,day_name,hour,minute,season,day_section,Public_Holiday,consumption,solar,alwaysOn,gridImport,gridExport,selfConsumption,selfSufficiency,active,reactive,voltages,phaseVoltages,currentHarmonics,voltageHarmonics
0,2020-03-15 18:15:00,1584288900000,2020,1,3,75,15,Sunday,18,15,winter,Evening,Non_Holiday,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[0.1, 3.7, 0.0, -81.6, -60.0, 30.2, -230.6, -0...","[0.5, 0.9, 0.0, 55.4, 132.4, 14.0, 137.5, 0.2,...","[244.5, None, None]","[244.5, None, None]","[[], [], [], [], [], [], [], [], [], [], [], [...","[[], [], [], [], [], [], [], [], [], [], [], [..."
1,2020-03-15 18:20:00,1584289200000,2020,1,3,75,15,Sunday,18,20,winter,Evening,Non_Holiday,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[0.1, 3.8, 0.0, -82.2, -58.5, 30.5, -233.1, -0...","[0.5, 0.9, 0.0, 55.7, 131.9, 14.3, 144.3, 0.2,...","[246.1, None, None]","[246.1, None, None]","[[], [], [], [], [], [], [], [], [], [], [], [...","[[], [], [], [], [], [], [], [], [], [], [], [..."
2,2020-03-15 18:25:00,1584289500000,2020,1,3,75,15,Sunday,18,25,winter,Evening,Non_Holiday,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[0.1, 28.5, -26.5, -74.4, -79.2, 55.5, -228.5,...","[0.5, 19.0, 8.8, 79.6, 137.6, 29.4, 175.0, 0.4...","[246.4, None, None]","[246.4, None, None]","[[], [], [], [], [], [], [], [], [], [], [], [...","[[], [], [], [], [], [], [], [], [], [], [], [..."
3,2020-03-15 18:30:00,1584289800000,2020,1,3,75,15,Sunday,18,30,winter,Evening,Non_Holiday,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[0.5, 21.3, -18.6, -76.9, -54.7, 48.7, -227.8,...","[2.2, 13.3, 6.5, 72.4, 135.9, 24.8, 152.8, 0.8...","[246.0, None, None]","[246.0, None, None]","[[], [], [], [], [], [], [], [], [], [], [], [...","[[], [], [], [], [], [], [], [], [], [], [], [..."
4,2020-03-15 18:35:00,1584290100000,2020,1,3,75,15,Sunday,18,35,winter,Evening,Non_Holiday,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[0.1, 3.7, 0.0, -82.6, -33.2, 30.7, 96.6, 0.2,...","[0.5, 0.9, 0.0, 55.2, 128.1, 13.9, 87.7, 0.7, ...","[246.2, None, None]","[246.2, None, None]","[[], [], [], [], [], [], [], [], [], [], [], [...","[[], [], [], [], [], [], [], [], [], [], [], [..."


HOURLY DATA WRANGLING

In [118]:
df_hourly.sample(5)

,date,timestamp,consumption,solar,alwaysOn,gridImport,gridExport,selfConsumption,selfSufficiency,active,reactive,voltages,phaseVoltages,currentHarmonics,voltageHarmonics
5041,2020-10-12 16:00:00,1602511200000,4062.020,877.101,1816.844,3184.919,0.0,100.0,21.59,"[1.2, 43.4, 0.0, None, None, None, None, 3.6, ...","[4.9, 10.6, 0.0, None, None, None, None, 2.6, ...","[242.2, None, None]","[242.2, None, None]","[[], [], [], [], [], [], [], [], [], [], [], [...","[[], [], [], [], [], [], [], [], [], [], [], [..."
7015,2021-01-02 22:00:00,1609617600000,2026.002,706.208,1673.982,1319.894,0.0,100.0,34.85,"[1.5, 42.3, -0.9, None, None, None, None, 5.9,...","[-6.3, 9.6, 0.0, None, None, None, None, -2.8,...","[239.4, None, None]","[239.4, None, None]","[[], [], [], [], [], [], [], [], [], [], [], [...","[[], [], [], [], [], [], [], [], [], [], [], [..."
1955,2020-06-06 02:00:00,1591401600000,2028.896,711.561,1599.615,1317.334,0.0,100.0,35.07,"[1.2, 43.2, 0.0, None, None, None, None, 4.7, ...","[4.8, 9.6, 0.0, None, None, None, None, 2.6, N...","[241.0, None, None]","[241.0, None, None]","[[], [], [], [], [], [], [], [], [], [], [], [...","[[], [], [], [], [], [], [], [], [], [], [], [..."
6927,2020-12-30 06:00:00,1609300800000,2085.770,753.318,1680.025,1332.251,0.0,100.0,36.13,"[1.5, 42.3, 0.0, None, None, None, None, 5.9, ...","[-6.2, 9.6, 0.1, None, None, None, None, -2.8,...","[239.2, None, None]","[239.2, None, None]","[[], [], [], [], [], [], [], [], [], [], [], [...","[[], [], [], [], [], [], [], [], [], [], [], [..."
6734,2020-12-22 05:00:00,1608606000000,2171.273,753.696,1916.432,1417.476,0.0,100.0,34.72,"[1.2, 44.1, 0.0, None, None, None, None, 4.8, ...","[-5.6, 10.8, 0.3, None, None, None, None, -2.7...","[243.9, None, None]","[243.9, None, None]","[[], [], [], [], [], [], [], [], [], [], [], [...","[[], [], [], [], [], [], [], [], [], [], [], [..."


In [119]:
print(do_data_information(df_hourly))

--------------------------------------------------------------------
First rows
                  date  ...                                   voltageHarmonics
0  2020-03-15 18:00:00  ...  [[], [], [], [], [], [], [], [], [], [], [], [...
1  2020-03-15 19:00:00  ...  [[], [], [], [], [], [], [], [], [], [], [], [...
2  2020-03-15 20:00:00  ...  [[], [], [], [], [], [], [], [], [], [], [], [...
3  2020-03-15 21:00:00  ...  [[], [], [], [], [], [], [], [], [], [], [], [...
4  2020-03-15 22:00:00  ...  [[], [], [], [], [], [], [], [], [], [], [], [...

[5 rows x 15 columns]
--------------------------------------------------------------------
Dimension of the data
This data has 7979 rows and 15 variables
None


In [120]:
print(check_missing_values_table(df_hourly))

Your selected dataframe has 15 columns.
There are 0 columns that have missing values.
Empty DataFrame
Columns: [Missing Values, % of Total Values]
Index: []


In [121]:
print(do_nunique_value(df_hourly))

The Numver of unique values per columns are as follows
                  nunique
date                 7979
timestamp            7979
consumption          7963
solar                7876
alwaysOn              714
gridImport           7946
gridExport              1
selfConsumption         2
selfSufficiency      3092
active               7979
reactive             7979
voltages              280
phaseVoltages         280
currentHarmonics        1
voltageHarmonics        1
None


In [122]:
df_hourly=do_datepart_extraction(df_hourly)

In [123]:
df_hourly.date.min(),df_hourly.date.max()

(Timestamp('2020-03-15 18:00:00'), Timestamp('2021-02-12 01:00:00'))

In [124]:
#checking for days where there weere no consumptions
df_hourly[df_hourly.consumption==0].value_counts().count(),df_hourly[df_hourly.consumption==0].value_counts()

(1,
 date                 timestamp      consumption  solar  alwaysOn  gridImport  gridExport  selfConsumption  selfSufficiency  active                                                                                                                                                                          reactive                                                                                                                                                                    voltages             phaseVoltages        currentHarmonics                                                                                                  voltageHarmonics                                                                                                  dateOnly    year  quarter  month  weekday  day_name  dayofyear  day  hour  minute  season  day_section
 2020-03-15 18:00:00  1584288000000  0.0          0.0    0.0       0.0         0.0         0.0              0.0              [1.3, 78.9, -45.1, -297.3

In [126]:
#checking for days where there weere  consumptions
df_hourly[df_hourly.consumption>0].value_counts().count(),df_hourly[df_5min.consumption>0].value_counts()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


(7978,
 date                 timestamp      consumption  solar    alwaysOn  gridImport  gridExport  selfConsumption  selfSufficiency  active                                                                                                                                                                     reactive                                                                                                                                                                     voltages             phaseVoltages        currentHarmonics                                                                                                  voltageHarmonics                                                                                                  dateOnly    year  quarter  month  weekday  day_name  dayofyear  day  hour  minute  season  day_section  
 2021-02-12 01:00:00  1613084400000  1970.954     555.986  1006.751  1415.067    0.0         100.0            28.20            [1.5, 42.9, -1.2, No

In [127]:
df_hourly['Public_Holiday'] = df_hourly['dateOnly'].map(mapping)

In [128]:
df_hourly['Public_Holiday']=df_hourly['Public_Holiday'].apply(lambda x:np.NaN if isinstance(x, str) and (x.isspace() or not x) else x).replace(np.NaN,'Non_Holiday')


In [129]:
df_hourly.sample(5)

,date,timestamp,consumption,solar,alwaysOn,gridImport,gridExport,selfConsumption,selfSufficiency,active,reactive,voltages,phaseVoltages,currentHarmonics,voltageHarmonics,dateOnly,year,quarter,month,weekday,day_name,dayofyear,day,hour,minute,season,day_section,Public_Holiday
657,2020-04-12 03:00:00,1586653200000,3373.360,470.453,805.169,2902.906,0.0,100.0,13.95,"[1.2, 44.1, -0.3, None, None, None, None, 3.6,...","[6.0, 10.8, 0.0, None, None, None, None, 2.6, ...","[243.3, None, None]","[243.3, None, None]","[[], [], [], [], [], [], [], [], [], [], [], [...","[[], [], [], [], [], [], [], [], [], [], [], [...",2020-04-12,2020,2,4,6,Sunday,103,12,3,0,winter,Late Night,Easter Sunday
5677,2020-11-08 04:00:00,1604800800000,2337.120,854.753,1733.989,1482.366,0.0,100.0,36.57,"[1.5, 42.0, 0.0, None, None, None, None, 4.1, ...","[-6.2, 9.6, 0.0, None, None, None, None, -2.5,...","[238.7, None, None]","[238.7, None, None]","[[], [], [], [], [], [], [], [], [], [], [], [...","[[], [], [], [], [], [], [], [], [], [], [], [...",2020-11-08,2020,4,11,6,Sunday,313,8,4,0,winter,Late Night,Non_Holiday
4274,2020-09-10 17:00:00,1599750000000,3446.951,660.678,1552.768,2785.973,0.0,100.0,19.18,"[1.4, 43.3, 0.0, None, None, None, None, 4.8, ...","[5.9, 10.4, 0.0, None, None, None, None, 3.1, ...","[241.8, None, None]","[241.8, None, None]","[[], [], [], [], [], [], [], [], [], [], [], [...","[[], [], [], [], [], [], [], [], [], [], [], [...",2020-09-10,2020,3,9,3,Thursday,254,10,17,0,winter,Evening,Non_Holiday
1518,2020-05-18 00:00:00,1589752800000,1855.298,745.436,1571.987,1109.961,0.0,100.0,40.17,"[1.5, 43.2, 0.0, None, None, None, None, 4.7, ...","[6.7, 9.6, 0.0, None, None, None, None, 2.5, N...","[241.6, None, None]","[241.6, None, None]","[[], [], [], [], [], [], [], [], [], [], [], [...","[[], [], [], [], [], [], [], [], [], [], [], [...",2020-05-18,2020,2,5,0,Monday,139,18,0,0,spring,Late Night,Non_Holiday
4745,2020-09-30 08:00:00,1601445600000,3816.799,934.287,1292.415,2882.511,0.0,100.0,24.48,"[1.5, 41.8, 0.0, None, None, None, None, 4.7, ...","[6.2, 9.6, 0.0, None, None, None, None, 3.1, N...","[237.9, None, None]","[237.9, None, None]","[[], [], [], [], [], [], [], [], [], [], [], [...","[[], [], [], [], [], [], [], [], [], [], [], [...",2020-09-30,2020,3,9,2,Wednesday,274,30,8,0,winter,Early Morning,Non_Holiday


In [130]:
df_hourly['Public_Holiday'].unique()

array(['Non_Holiday', 'Good Friday', 'Holy Saturday', 'Easter Sunday',
       'Easter Monday', 'Independence Day', "Worker's Day", 'Africa Day',
       "Heroes' Day", 'Zimbabwe Defence Forces Day', 'National Unity Day',
       'Christmas Day', 'Boxing Day', "New Year's Day"], dtype=object)

In [131]:
df_hourly=df_hourly[['date','timestamp','year','quarter','month','dayofyear','day','day_name','hour','minute','season','day_section','Public_Holiday','consumption','solar','alwaysOn','gridImport','gridExport','selfConsumption','selfSufficiency','active','reactive','voltages','phaseVoltages','currentHarmonics','voltageHarmonics']]

DAILY WRANGLING

In [144]:
print(do_data_information(df_daily))




--------------------------------------------------------------------
First rows
        date  ...                                   voltageHarmonics
0 2020-03-15  ...  [[], [], [], [], [], [], [], [], [], [], [], [...
1 2020-03-16  ...  [[], [], [], [], [], [], [], [], [], [], [], [...
2 2020-03-17  ...  [[], [], [], [], [], [], [], [], [], [], [], [...
3 2020-03-18  ...  [[], [], [], [], [], [], [], [], [], [], [], [...
4 2020-03-19  ...  [[], [], [], [], [], [], [], [], [], [], [], [...

[5 rows x 26 columns]
--------------------------------------------------------------------
Dimension of the data
This data has 335 rows and 26 variables
None


In [145]:
print(check_missing_values_table(df_daily))

Your selected dataframe has 26 columns.
There are 0 columns that have missing values.
Empty DataFrame
Columns: [Missing Values, % of Total Values]
Index: []


In [146]:
print(do_nunique_value(df_daily))

The Numver of unique values per columns are as follows
                  nunique
date                  335
timestamp             335
year                    2
quarter                 4
month                  12
dayofyear             335
day                    31
day_name                7
hour                    1
minute                  1
season                  4
day_section             1
Public_Holiday         14
consumption           335
solar                 333
alwaysOn              333
gridImport            335
gridExport              1
selfConsumption         2
selfSufficiency       310
active                335
reactive              335
voltages               95
phaseVoltages          95
currentHarmonics        1
voltageHarmonics        1
None


In [147]:
#Feature Engineering
df_daily=do_datepart_extraction(df_daily)

In [138]:
df_daily.date.min(),df_daily.date.max()

(Timestamp('2020-03-15 00:00:00'), Timestamp('2021-02-12 00:00:00'))

In [139]:
df_daily.consumption.min(),df_daily.consumption.max()

(3482.0, 122179.38)

In [140]:
df_daily[df_daily.consumption==0].value_counts().count(),df_daily[df_daily.consumption==0].value_counts()

(0, Series([], dtype: int64))

In [141]:
df_daily['Public_Holiday'] =df_daily['dateOnly'].map(mapping)

In [142]:
df_daily['Public_Holiday']=df_daily['Public_Holiday'].apply(lambda x:np.NaN if isinstance(x, str) and (x.isspace() or not x) else x).replace(np.NaN,'Non_Holiday')

In [143]:
df_daily=df_daily[['date','timestamp','year','quarter','month','dayofyear','day','day_name','hour','minute','season','day_section','Public_Holiday','consumption','solar','alwaysOn','gridImport','gridExport','selfConsumption','selfSufficiency','active','reactive','voltages','phaseVoltages','currentHarmonics','voltageHarmonics']]

MonthLy Wrangling

In [148]:
print(do_data_information(df_monthly))



--------------------------------------------------------------------
First rows
         date  ...                                   voltageHarmonics
0  2020-03-01  ...  [[], [], [], [], [], [], [], [], [], [], [], [...
1  2020-04-01  ...  [[], [], [], [], [], [], [], [], [], [], [], [...
2  2020-05-01  ...  [[], [], [], [], [], [], [], [], [], [], [], [...
3  2020-06-01  ...  [[], [], [], [], [], [], [], [], [], [], [], [...
4  2020-07-01  ...  [[], [], [], [], [], [], [], [], [], [], [], [...

[5 rows x 15 columns]
--------------------------------------------------------------------
Dimension of the data
This data has 12 rows and 15 variables
None


In [149]:
print(check_missing_values_table(df_monthly))

Your selected dataframe has 15 columns.
There are 0 columns that have missing values.
Empty DataFrame
Columns: [Missing Values, % of Total Values]
Index: []


In [150]:
print(do_nunique_value(df_monthly))

The Numver of unique values per columns are as follows
                  nunique
date                   12
timestamp              12
consumption            12
solar                  12
alwaysOn               12
gridImport             12
gridExport              1
selfConsumption         1
selfSufficiency        12
active                 12
reactive               12
voltages               10
phaseVoltages          10
currentHarmonics        1
voltageHarmonics        1
None


In [151]:
#Feature Engineering
df_monthly=do_datepart_extraction(df_monthly)

In [152]:
df_monthly.date.min(),df_monthly.date.max()

(Timestamp('2020-03-01 00:00:00'), Timestamp('2021-02-01 00:00:00'))

In [153]:
df_monthly.consumption.min(),df_monthly.consumption.max()

(928220.438, 2535679.307)

In [154]:
#checking for days where there weere no consumptions
df_monthly[df_daily.consumption==0].value_counts().count(),df_monthly[df_monthly.consumption==0].value_counts()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


(0, Series([], dtype: int64))

In [155]:
df_monthly['Public_Holiday'] =df_monthly['dateOnly'].map(mapping)

In [156]:
df_monthly['Public_Holiday']=df_monthly['Public_Holiday'].apply(lambda x:np.NaN if isinstance(x, str) and (x.isspace() or not x) else x).replace(np.NaN,'Non_Holiday')

In [157]:
df_monthly=df_monthly[['date','timestamp','year','quarter','month','dayofyear','day','day_name','hour','minute','season','day_section','Public_Holiday','consumption','solar','alwaysOn','gridImport','gridExport','selfConsumption','selfSufficiency','active','reactive','voltages','phaseVoltages','currentHarmonics','voltageHarmonics']]

In [158]:
#to CSV
df_5min.to_csv('NeedEnergyAPI_5min.csv',index=False)
df_hourly.to_csv('NeedEnergyAPI_hourly.csv',index=False)
df_daily.to_csv('NeedEnergyAPI_daily.csv',index=False)
df_monthly.to_csv('NeedEnergyAPI_monthly.csv',index=False)


#DATA QUALITY CHECK:
*Data summary:
* Raw_NeedEnergyApi_5min :
Raw_Records: 95649 with 15 columns
Cleaned(Current) :95649,

* Raw_NeedEnergyApi_hourly :
Raw_Record :7979 with 15 columns
Cleaned(Current) :7979,
* Raw_NeedEnergyApi_Daily :
Raw_Records: 335 with 15 columns
Cleaned (current):335
* Raw_NeedEnergyApi_Monthly :
Raw_records:12 with 15 columns
cleaned =12
* BASED ON THE FOLLOWING QUALITY DIMENSIONS:(ACCURACY,COMPELETENESS,VALUES FREE FROM CONTRADICTION)
THE NEEDENERGY API 5MIN,HOURLY,DAILTY,MONTHLY DATASET has zero missing values.More data required for predictive analysis,

## Feature Engineering:
More features were extracted and created from the date column feature:
* New features Generated :day,hour,month,year,day_name,
seasons(winter,autumn,spring),day sections(Night,Evening,Afternoon)

* Others:Jupyter starter files for 
Generalized Helper functions were created to generate seasons,day_sections,day,month,year,hour,quarter,were created for use for anyone willing to start-up cleaning or other task activities

* In-Progress Helper function
Public-holiday and many more for solar-iiradiance dataset.
